In [ ]:
import socket
import io
import os
import requests
from google.cloud import vision_v1


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ezz-startup-ocr-a81700d3a74a.json"
client = vision_v1.ImageAnnotatorClient()

# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# get local machine name
host = '192.168.1.10'
port = 9999

# bind the socket to a public host, and a well-known port
serversocket.bind((host, port))

# become a server socket
serversocket.listen(1)

while True:
    # establish a connection
    clientsocket, addr = serversocket.accept()

    print("Got a connection from %s" % str(addr))

    # receive data from the client
    data = clientsocket.recv(1024).decode('utf-8')
    print("Received data: %s" % data)
    
    response = requests.get(data)
    content = response.content

    image = vision_v1.types.Image(content=content)

    image_context = vision_v1.types.ImageContext(language_hints=['en'])
    response = client.document_text_detection(image=image, image_context=image_context)

    document = response.full_text_annotation

    message = ''
    
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    text = ''.join([symbol.text for symbol in word.symbols])
                    message += text

    # send a response back to the client
    clientsocket.send(message.encode('utf-8'))

    # close the client connection
    clientsocket.close()